In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
cd /content/drive/My Drive/Project_HonesDoctor2020

/content/drive/My Drive/Project_HonesDoctor2020


In [3]:
!pip install googletrans
!pip install tqdm #--upgrade
!pip install twython

  Created wheel for googletrans: filename=googletrans-2.4.0-cp36-none-any.whl size=15777 sha256=d3d8b76a62bbf45a49cb55e4808cc277d997bc087cf7f649a0e91626caa769a6
  Stored in directory: /root/.cache/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [0]:
import requests
from bs4 import BeautifulSoup
from time import  time, sleep
from random import randint

from googletrans import Translator

from tqdm import tqdm
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from nltk import sent_tokenize, word_tokenize
from nltk.sentiment.vader import  SentimentIntensityAnalyzer
from nltk.sentiment.util import mark_negation

In [6]:
import nltk

nltk.download("punkt")
nltk.download("vader_lexicon")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [0]:
def honestdoc_comment(url):
  """
    INPUT
      url : String
        URL of the target website
    OUTPUT
      comment : List
        List of comments
      score : List
        List of rating score
  """
  data = requests.get(url)
  print("requests code : {}".format(data.status_code))
  print("note\n2xx : successs\n4xx, 5xx : error")

  start_time = time()
  r = requests.get(url, params=dict(query="web scraping", page=1))
  soup = BeautifulSoup(r.text, "html.parser")
  n = len(soup.find_all("div", {"class": "comments__content"}))

  comment = [ soup.find_all("div", {"class": "comments__content"})[i].get_text().strip() for i in range(0, n) ]
  score = [ soup.find_all("span", {"class": "stars star-rating"})[i].attrs["data-score"] for i in range(0, n) ]

  elapsed_time = time() - start_time
  print("Time used for scraping data from page 1 : {} s".format(elapsed_time))
  sleep(randint(1,3))

  p = 2
  while n > 0:
    start_time = time()
    r = requests.get(url, params=dict(query="web scraping", page=p))
    soup = BeautifulSoup(r.text, "html.parser")
    n = len(soup.find_all("div", {"class": "comments__content"}))
    [ comment.append(soup.find_all("div", {"class": "comments__content"})[i].get_text().strip()) for i in range(0,n) ]
    [ score.append(soup.find_all("span", {"class": "stars star-rating"})[i].attrs["data-score"]) for i in range(0,n) ]
    elapsed_time = time() - start_time
    print("Time used for scraping data from page {} : {} s".format(p, elapsed_time))
    p += 1
    sleep(randint(1,3))
  #backup
  pd.DataFrame({"comment": comment, "score": score}).to_csv("comment_"+str(url[url.rfind("/")+1:]) + ".csv", index=False)
  return comment, score


In [0]:
# We choose the following lists of hospitals to make sentiment analysis.
# โรงพยาบาลศิริราช ปิยมหาราชการุณย์
# https://www.honestdocs.co/hospitals/siriraj-piyamaharajkarun-hospital
# โรงพยาบาลสิรินธร
# https://www.honestdocs.co/hospitals/sirindhorn-hospital
# โรงพยาบาลตำรวจ
# https://www.honestdocs.co/hospitals/police-general-hospital

In [8]:
#โรงพยาบาลตำรวจ
police_comments, police_scores = honestdoc_comment(r"https://www.honestdocs.co/hospitals/police-general-hospital")

requests code : 200
note
2xx : successs
4xx, 5xx : error
Time used for scraping data from page 1 : 0.5034072399139404 s
Time used for scraping data from page 2 : 0.3772139549255371 s
Time used for scraping data from page 3 : 0.37371277809143066 s
Time used for scraping data from page 4 : 0.42806458473205566 s
Time used for scraping data from page 5 : 0.3701767921447754 s
Time used for scraping data from page 6 : 0.368746280670166 s
Time used for scraping data from page 7 : 0.46930813789367676 s
Time used for scraping data from page 8 : 0.3905036449432373 s
Time used for scraping data from page 9 : 0.3755311965942383 s
Time used for scraping data from page 10 : 0.3678162097930908 s
Time used for scraping data from page 11 : 0.3703272342681885 s
Time used for scraping data from page 12 : 0.3712005615234375 s
Time used for scraping data from page 13 : 0.37633371353149414 s
Time used for scraping data from page 14 : 0.36984872817993164 s
Time used for scraping data from page 15 : 0.3794891

In [9]:
police_comments

['ใช้สิทธิ์ประกันสังคมค่ะ รอไม่นานแถมคุณหมอและพยาบาลบริการดีมากค่ะ พูดจาดีให้รายละเอียดครบ ยังไม่เคยใช้บริการทุกแผนกนะคะแต่ แผนก สูตินารีเวช ดีมากค่ะ อยากย้ายประกันสังคมกลับมาที่นี่มากๆเลยค่ะ เดินทางสะดวก อยู่ใกล้ห้าง ออกมาเดินเล่นระหว่างรอผลตรวจได้แบบชิลๆเลย ถึงตอนนี้ประกันสังคมไม่ได้อยู่ที่นี่แล้วดิฉันก็มาใช้บริการเหมือนเดิมค่ะ',
 'ขอชื่นชมคุณหมอและคุณพยาบาลโรงบาลตำรวจมากๆเลยค่ะที่ช่วยเหลือสามีจากการผ่าตัดใหญ่ครั้งนี้เราต้องขอยอมรับในฝีมือของคุณหมอจริงๆผลออกมาดีสามีเราปลอดภัยดีเพราะคุณหมอและคุณพยาบาลใจมากถึงไม่มีผลกระทบอะไรต่อสามีเราทำให้เรารู้ทันทีเลยว่าเราคิดถูกแล้วที่เลือกและตัดสินใจให้สามีผ่าตัดที่โรงบาลตำรวจตอนนี้สามีนอนรักษาตัวห้องพิเศษคุณพยาบาลน่ารักทุกคนเดินเข้ามาดูสามีเราตลอดๆพูดจาดีทุกคน\r\n\r\nแต่ก็ขอฝากถึงแม่บ้านทำความสะอาดในห้องผู้ป่วยควรใส่ใจห้องพักของผู้ป่วยหน่อยคือตรงนี้เราก็ไม่โอเคเท่าไรคือทำแบบรีบๆยังไงก็ไม่รู้จนเรามองดูแล้วว่ายังไม่สะอาดเราเลยเดินไปเอาไม้กวาดมาเก็บกวาดเองรู้ป่ะว่าขี้ผงและเศษขยะเล็กๆที่อยู่ใต้เตียงคนเฝ้าใต้เตียงผู้ป่วยและมุมห้องทุกๆมุมเรากวาดได้เยอะ

In [0]:
'''
gt = Translator()
input_text = "หิวข้าวรึยัง"
gt.translate(input_text, src="th", dest="en").text
'''

In [0]:
def th2en(comment):
  print(comment + "\n")
  return Translator().translate(comment, src="th", dest="en").text

In [13]:
police = pd.read_csv("comment_police-general-hospital.csv")
print(police.shape)
police.head()

(262, 2)


,comment,score
0,ใช้สิทธิ์ประกันสังคมค่ะ รอไม่นานแถมคุณหมอและพย...,5
1,ขอชื่นชมคุณหมอและคุณพยาบาลโรงบาลตำรวจมากๆเลยค่...,5
2,ลุงผมเป็นคนไข้ ขณะนี้เวลา 13.20น. แล้ว ข้าวเที...,1
3,จากผู้รับการรักษาจริงอยากบอกว่า รพ.ตำรวจระบบจั...,2
4,ลูกชายนอนห้องพิเศษไม่มีพยาบาลเข้ามาดูแลเลย,2


In [0]:
#for check lenght with issue comment
i = 0
for x in police['comment']:
 if len(police['comment'][i]) > 1000:
  print(len(police['comment'][i]), "-> index : ", i)
 i=i+1

In [15]:
tqdm.pandas()

police["en"] = police.progress_apply(lambda x: th2en(x["comment"]), axis=1)

  1%|          | 2/262 [00:00<00:15, 16.28it/s]

ใช้สิทธิ์ประกันสังคมค่ะ รอไม่นานแถมคุณหมอและพยาบาลบริการดีมากค่ะ พูดจาดีให้รายละเอียดครบ ยังไม่เคยใช้บริการทุกแผนกนะคะแต่ แผนก สูตินารีเวช ดีมากค่ะ อยากย้ายประกันสังคมกลับมาที่นี่มากๆเลยค่ะ เดินทางสะดวก อยู่ใกล้ห้าง ออกมาเดินเล่นระหว่างรอผลตรวจได้แบบชิลๆเลย ถึงตอนนี้ประกันสังคมไม่ได้อยู่ที่นี่แล้วดิฉันก็มาใช้บริการเหมือนเดิมค่ะ

ขอชื่นชมคุณหมอและคุณพยาบาลโรงบาลตำรวจมากๆเลยค่ะที่ช่วยเหลือสามีจากการผ่าตัดใหญ่ครั้งนี้เราต้องขอยอมรับในฝีมือของคุณหมอจริงๆผลออกมาดีสามีเราปลอดภัยดีเพราะคุณหมอและคุณพยาบาลใจมากถึงไม่มีผลกระทบอะไรต่อสามีเราทำให้เรารู้ทันทีเลยว่าเราคิดถูกแล้วที่เลือกและตัดสินใจให้สามีผ่าตัดที่โรงบาลตำรวจตอนนี้สามีนอนรักษาตัวห้องพิเศษคุณพยาบาลน่ารักทุกคนเดินเข้ามาดูสามีเราตลอดๆพูดจาดีทุกคน

แต่ก็ขอฝากถึงแม่บ้านทำความสะอาดในห้องผู้ป่วยควรใส่ใจห้องพักของผู้ป่วยหน่อยคือตรงนี้เราก็ไม่โอเคเท่าไรคือทำแบบรีบๆยังไงก็ไม่รู้จนเรามองดูแล้วว่ายังไม่สะอาดเราเลยเดินไปเอาไม้กวาดมาเก็บกวาดเองรู้ป่ะว่าขี้ผงและเศษขยะเล็กๆที่อยู่ใต้เตียงคนเฝ้าใต้เตียงผู้ป่วยและมุมห้องทุกๆมุมเรากวาดได้เยอะเลยล่ะค่ะ  

  2%|▏         | 4/262 [00:00<00:23, 11.17it/s]

ลุงผมเป็นคนไข้ ขณะนี้เวลา 13.20น. แล้ว ข้าวเที่ยงยังไม่มา โทรตาม สามรอบแล้ว ห่วยแตกมากๆ

จากผู้รับการรักษาจริงอยากบอกว่า รพ.ตำรวจระบบจัดการแย่ โดยเฉพาะจุดทำประวัติใหม่ รอ2ชม. 9.30-11.30 ไม่เรียงคิวคนมากก่อนมาหลัง พอถามบอกระบบล้ม แต่คนอื่นถ่ายบัตรใหม่ส่งไปตรวจ นี่ถามบอกประกันสังคม คืออะไร? ไม่อยากรับปกส.หรอ ได้ตรวจ11.30พยาบาลบอกเร็วๆเลย หมอรับถึงแค่ เที่ยง พอจะตรวจอีกแผนกบอกหมอไม่มีช่วงบ่าย พอจะไปทำบัตรนัดจะมาวันอื่น บอกวันนี้มีหมอออกตรวจ ไล่ไปเอาแฟ้มที่แผนกเดิม คือต้องเดินไป-กลับระหว่าง 2 ตึกเอง 2 รอบ คนป่วยนะคะ แต่คุณหมอที่นี่ยอมรับเลยค่ะ ดีมาก ทั้ง 2 แผนกที่ไปตรวจ ดียิ่งกว่ารพ.เอกชนบางแห่งซะอีก ใส่ใจเรา ตรวจระเอียดและให้คำแนะนำดีมาก คือที่ทน เพราะหมอน่ารักจริงๆ นอกนั้นแย่มาก...จากใจคนใช้ประกันสังคม รพ.ตำรวจ



  2%|▏         | 6/262 [00:00<00:27,  9.31it/s]

...

100%|█████████▉| 261/262 [00:57<00:00,  1.24it/s]

เป็นโรงบาลที่ดีสะดวกรวดเร็วแบ่งแผนกผู้มารักษาได้ดีไม่ต้องกลัวรอนานเลยครับ



100%|██████████| 262/262 [00:58<00:00,  1.08it/s]

เจ้าหน้าให้การตอนรับดีมากเลยครับยิ้มแย้มแจ่มใสมากครับ



100%|██████████| 262/262 [00:59<00:00,  4.40it/s]


In [16]:
police.head(10)

,comment,score,en
0,ใช้สิทธิ์ประกันสังคมค่ะ รอไม่นานแถมคุณหมอและพย...,5,Use Social Security I can not wait for the doc...
1,ขอชื่นชมคุณหมอและคุณพยาบาลโรงบาลตำรวจมากๆเลยค่...,5,We appreciate your doctors and nurses in hospi...
2,ลุงผมเป็นคนไข้ ขณะนี้เวลา 13.20น. แล้ว ข้าวเที...,1,"My uncle is a patient currently at 13.20 hrs.,..."
3,จากผู้รับการรักษาจริงอยากบอกว่า รพ.ตำรวจระบบจั...,2,From the patient actually wants to say. Hospit...
4,ลูกชายนอนห้องพิเศษไม่มีพยาบาลเข้ามาดูแลเลย,2,Son sleeping room nurse took no care at all.
5,ตอนที่ไม่สบายหนักได้ไปรักษากับโรงพยาบาลหนึ่งมา...,5,The discomfort is hard to maintain the hospita...
6,ไม่ได้ฝากครรภ์ที่โรงพยาบาลตำรวจ แต่มีสิทธิ์ประ...,5,Police have no prenatal care at the hospital. ...
7,ไปฝากครรภ์ ที่โรงบาลตำรวจ รอคิวแปบเดียว แต่พยา...,3,"ANC to the hospital, police Wait a minute But ..."
8,เป็นโรงพยาบาลที่ดี ยอมรับในเรื่องหมอเก่ง แต่ก็...,4,A good hospital Accept the good doctor But sti...
9,คลอดที่ร.พตำรวจ ดูแลดี พูดจาดี,4,The birth process. The police maintained a gre...


In [21]:
rename_col = ['comment','en','score']
police = police.reindex(columns=rename_col)
police.columns = ['th_comment','en_comment','score']
police.head()

,th_comment,en_comment,score
0,ใช้สิทธิ์ประกันสังคมค่ะ รอไม่นานแถมคุณหมอและพย...,Use Social Security I can not wait for the doc...,5
1,ขอชื่นชมคุณหมอและคุณพยาบาลโรงบาลตำรวจมากๆเลยค่...,We appreciate your doctors and nurses in hospi...,5
2,ลุงผมเป็นคนไข้ ขณะนี้เวลา 13.20น. แล้ว ข้าวเที...,"My uncle is a patient currently at 13.20 hrs.,...",1
3,จากผู้รับการรักษาจริงอยากบอกว่า รพ.ตำรวจระบบจั...,From the patient actually wants to say. Hospit...,2
4,ลูกชายนอนห้องพิเศษไม่มีพยาบาลเข้ามาดูแลเลย,Son sleeping room nurse took no care at all.,2


In [0]:
#export to csv incluing th-comment, en-comment, score
police.to_csv('en-comment_police-general-hospital.csv', index=False ,encoding='utf-8')

In [25]:
en_police = pd.read_csv("en-comment_police-general-hospital.csv")
print(en_police.shape)
en_police.head()

(262, 3)


,th_comment,en_comment,score
0,ใช้สิทธิ์ประกันสังคมค่ะ รอไม่นานแถมคุณหมอและพย...,Use Social Security I can not wait for the doc...,5
1,ขอชื่นชมคุณหมอและคุณพยาบาลโรงบาลตำรวจมากๆเลยค่...,We appreciate your doctors and nurses in hospi...,5
2,ลุงผมเป็นคนไข้ ขณะนี้เวลา 13.20น. แล้ว ข้าวเที...,"My uncle is a patient currently at 13.20 hrs.,...",1
3,จากผู้รับการรักษาจริงอยากบอกว่า รพ.ตำรวจระบบจั...,From the patient actually wants to say. Hospit...,2
4,ลูกชายนอนห้องพิเศษไม่มีพยาบาลเข้ามาดูแลเลย,Son sleeping room nurse took no care at all.,2
